In [1]:
import sys
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
import torch
import torch
from torch import nn
from sklearn.model_selection import train_test_split
import random

print('python:', sys.version)
print('torch:', torch.__version__)
# print(torch.cuda.get_device_name(),'is available? ->', torch.cuda.is_available())
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
BATCHSIZE = 8
EPOCHS = 5
random.seed(10)

/home/eringi/kaggle/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


python: 3.10.11 (main, May  7 2023, 19:24:25) [Clang 16.0.3 ]
torch: 1.13.1+cu117


In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [3]:
cols = train.columns
cols = cols.drop(['Id', 'EJ', 'Class'])

train_quantitative = train[cols]
test_x = test[cols]

first_quartiles = train_quantitative.quantile(0.25)
third_quartiles = train_quantitative.quantile(0.75)

iqr = third_quartiles - first_quartiles

lower_bound = first_quartiles - (iqr * 1.5) #外れ値の下限
upper_bound = third_quartiles + (iqr * 1.5) #外れ値の上限

# train[cols] = train[(train[cols] > lower_bound) | (train[cols] < upper_bound)]
train_quantitative = train_quantitative.clip(lower_bound, upper_bound, axis=1)

In [4]:
x = train_quantitative
y = train['Class']
train_val_x, test_x, train_val_y, test_y = train_test_split(train_quantitative, y, train_size=0.8, shuffle=True, random_state=20)
train_x, val_x, train_y, val_y = train_test_split(train_val_x, train_val_y, train_size=0.8, shuffle=True, random_state=20)

In [5]:
dtrain = xgb.DMatrix(train_x, label=train_y)
dval = xgb.DMatrix(val_x, label=val_y)
dtest = xgb.DMatrix(test_x)

params = {
  'objective': 'binary:logistic',
  'silent':1, 
  'random_state':1234, 
  'eval_metric': 'rmse',
}
num_round = 100
watch_list = [(dtrain, 'train'), (dval, 'eval')]

In [6]:
# train
# xgb_model = xgb.train(
#     params,
#     dtrain,
#     num_round,
#     evals=watch_list,
#     early_stopping_rounds=10
# )

xgb_clf = XGBClassifier(
    learning_rate = 0.01,
    early_stopping_rounds = 10,
    tree_method = 'gpu_hist'
)
evals = [(val_x, val_y)]
xgb_clf.fit(train_x, train_y, eval_set=evals, verbose=True)

[0]	validation_0-logloss:0.68557
[1]	validation_0-logloss:0.67810
[2]	validation_0-logloss:0.67077
[3]	validation_0-logloss:0.66359
[4]	validation_0-logloss:0.65655
[5]	validation_0-logloss:0.64965
[6]	validation_0-logloss:0.64288
[7]	validation_0-logloss:0.63619
[8]	validation_0-logloss:0.62972
[9]	validation_0-logloss:0.62332
[10]	validation_0-logloss:0.61709
[11]	validation_0-logloss:0.61089
[12]	validation_0-logloss:0.60485
[13]	validation_0-logloss:0.59887
[14]	validation_0-logloss:0.59307
[15]	validation_0-logloss:0.58745
[16]	validation_0-logloss:0.58193
[17]	validation_0-logloss:0.57648
[18]	validation_0-logloss:0.57110
[19]	validation_0-logloss:0.56580
[20]	validation_0-logloss:0.56055
[21]	validation_0-logloss:0.55551
[22]	validation_0-logloss:0.55069
[23]	validation_0-logloss:0.54575
[24]	validation_0-logloss:0.54097
[25]	validation_0-logloss:0.53628
[26]	validation_0-logloss:0.53156
[27]	validation_0-logloss:0.52707
[28]	validation_0-logloss:0.52266
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=10,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [7]:
prob = xgb_clf.predict_proba(test_x)
preds = xgb_clf.predict(test_x)

num = len(preds)
acc_cnt = 0
for y, cls in zip(test_y, preds):
    if y == cls:
        acc_cnt += 1
  
print('acc:', acc_cnt / num)
    

acc: 0.8951612903225806


In [8]:

test_quantitative = test[cols]

first_quartiles = test_quantitative.quantile(0.25)
third_quartiles = test_quantitative.quantile(0.75)

iqr = third_quartiles - first_quartiles

lower_bound = first_quartiles - (iqr * 1.5) #外れ値の下限
upper_bound = third_quartiles + (iqr * 1.5) #外れ値の上限

# train[cols] = train[(train[cols] > lower_bound) | (train[cols] < upper_bound)]
test_quantitative = test_quantitative.clip(lower_bound, upper_bound, axis=1)

test_prob = xgb_clf.predict_proba(test_quantitative.to_numpy())

In [9]:
submit_df = pd.DataFrame(columns=['Id', 'class_0', 'class_1'])
submit_df['Id'] = test['Id']
submit_df['class_0'] = test_prob[:, 0]
submit_df['class_1'] = test_prob[:, 1]
submit_df.to_csv('submission.csv', index=False)